# Сбор данных для анализа тендеров

Источник: [https://zakupki.gov.ru](https://zakupki.gov.ru/epz/main/public/home.html)

### Импорт библиотек

In [226]:
# библиотеки для работы с HTML
import requests
import selenium
from selenium import webdriver
from bs4 import BeautifulSoup

# библиотеки для предобработки и анализа данных 
import pandas as pd

# библиотеки для работы в jupyter notebook
import warnings
warnings.filterwarnings(action='once')  # показывает предупреждения один раз
warnings.filterwarnings('ignore')

In [240]:
s = '16568666,6764453'

In [241]:
to_float(s)

16568666.6764453

In [238]:
def to_float(string):
    numbers = string.split(',')
    integer = int(numbers[0])
    fractional = int(numbers[1])
    return integer + fractional / 10 ** len(numbers[1])

###### Читаем страницу

In [5]:
URL = 'https://zakupki.gov.ru/epz/order/extendedsearch/rss.html?searchString=разработка+программного+обеспечения&morphology=on'

###### Создаем драйвер для браузера

In [205]:
# https://chromedriver.chromium.org/downloads

options = webdriver.ChromeOptions()
options.binary_location = '/Applications/Yandex.app/Contents/MacOS/Yandex'

webdriver_address = '../../drivers/chromedriver'

###### Тестируем подключение

In [207]:
driver = webdriver.Chrome(webdriver_address, options=options)
driver.get(URL)

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 79


In [198]:
html = driver.page_source

In [199]:
html = html.replace('&lt;', '<') \
           .replace('&gt;', '>') \
           .replace('&ndash;', '–') \
           .replace('&amp;', '&') 

In [210]:
soup = BeautifulSoup(html, 'lxml')

In [211]:
items = soup.find_all('item')

In [212]:
with open('data/2020-05-20_zakupki.txt', 'w', encoding='utf-8') as f:
    f.write(str(items))

In [213]:
with open('data/2020-05-20_zakupki.txt', 'r', encoding='utf-8') as f:
    h = f.read()

In [214]:
soup = BeautifulSoup(h, 'lxml')

In [215]:
items = soup.find_all('item')

In [216]:
list_of_dicts = []

In [217]:
item_dict = dict(
                 title='',
                 description='',   
                 price='',
                 date='',
                 law='',
                 author='',
                 status='',
                 link='',
                 query='',   
                 )

In [218]:
item_dict

{'title': '',
 'description': '',
 'price': '',
 'date': '',
 'law': '',
 'author': '',
 'status': '',
 'link': '',
 'query': ''}

In [219]:
items[1]

<item>
<title>Прочие №32009143442</title>
<link/>https://zakupki.gov.ru/223/purchase/public/purchase/info/common-info.html?regNumber=32009143442
      <description><b>Параметры поиска: </b><br/><strong>Искомое слово (словосочетание): </strong>разработка программного обеспечения<br/><strong>Закупки по: </strong>223-ФЗ и 44-ФЗ<br/><strong>Этап закупки: </strong>Подача заявок, Работа комиссии, Закупка завершена, Закупка отменена<br/><br/><strong>Найденный результат:</strong><br/><strong>Прочие</strong> <a href="https://zakupki.gov.ru/223/purchase/public/purchase/info/common-info.html?regNumber=32009143442" target="_blank"><strong>№ </strong><strong>32009143442</strong></a><br/><strong>Наименование объекта закупки: </strong>Услуги по разработке программного обеспечения ПТК СПВБ для обеспечения размещения ОФЗ и ОБР на новой ИТ-платформе АО СПВБ<br/><strong>Размещение выполняется по: </strong>223-ФЗ<br/><strong>Наименование Заказчика: </strong>АКЦИОНЕРНОЕ ОБЩЕСТВО "САНКТ-ПЕТЕРБУРГСКАЯ ВАЛЮТН

In [220]:
def find(string, start, end):
    i = string.find(start) + len(start)
    j = string.find(end)
    return string[i:j].strip()

In [221]:
for item in items:
    if item.text != '':
        __dict = dict()
        __dict['title'] = item.find('title').text
        __dict['description'] = find(item.find('description').text,
                                                  'Наименование объекта закупки:',
                                                  'Размещение выполняется по:',
                                                  )
        __dict['price'] = find(item.find('description').text,
                                                'Начальная цена контракта:',
                                                'Валюта:',
                                                )
        __dict['date'] = item.find('pubdate').text
        __dict['law'] = find(item.find('description').text,
                                          'Размещение выполняется по: ',
                                          'Наименование Заказчика:',
                                          )
        __dict['author'] = item.find('author').text
        __dict['status'] = find(item.find('description').text,
                                             'Этап размещения: ',
                                             'Идентификационный код закупки',
                                             )
        __dict['link'] = item.find('a')['href']
        __dict['query'] = find(item.find('description').text,
                                                  'Искомое слово (словосочетание):',
                                                  'Закупки по:',
                                                  )
        list_of_dicts.append(__dict)

In [222]:
len(list_of_dicts)

200

In [225]:
pd.DataFrame(list_of_dicts)

,title,description,price,date,law,author,status,link,query
0,Прочие №32009165595,ЗП – 81028028 (Выполнение работ по разработке ...,Многолотовый заказРазмещено: 20.05.2020Обновле...,"Wed, 20 May 2020 11:33:24 GMT",223-ФЗ,"ПУБЛИЧНОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""СБЕРБАНК РОССИИ""",Подача заявок,https://zakupki.gov.ru/223/purchase/public/pur...,разработка программного обеспечения
1,Прочие №32009143442,Услуги по разработке программного обеспечения ...,186575000.00,"Tue, 12 May 2020 20:54:22 GMT",223-ФЗ,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""САНКТ-ПЕТЕРБУРГСКАЯ ВАЛЮ...",Подача заявок,https://zakupki.gov.ru/223/purchase/public/pur...,разработка программного обеспечения
2,Прочие №32009133042,"Оказание услуг, по выполнению составной части ...",2000000.00,"Wed, 06 May 2020 18:02:25 GMT",223-ФЗ,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ОБРАЗОВА...,Закупка завершена,https://zakupki.gov.ru/223/purchase/public/pur...,разработка программного обеспечения
3,Прочие №32009031986,Право на заключение Договора на оказание услуг...,98474400.00,"Wed, 25 Mar 2020 14:37:01 GMT",223-ФЗ,"ПУБЛИЧНОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""МЕЖРЕГИОНАЛЬНА...",Работа комиссии,https://zakupki.gov.ru/223/purchase/public/pur...,разработка программного обеспечения
4,Прочие №32009150538,Право заключения договора на Разработка и внед...,13450960.00,"Thu, 14 May 2020 14:21:56 GMT",223-ФЗ,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""РОССИЙСКИЙ КОНЦЕРН ПО ПР...",Подача заявок,https://zakupki.gov.ru/223/purchase/public/pur...,разработка программного обеспечения
...,...,...,...,...,...,...,...,...,...
195,Электронный аукцион №0330300088120000001,Услуги по проектированию и разработке информац...,6260000.00,"Fri, 17 Jan 2020 10:38:45 GMT",44-ФЗ,УПРАВЛЕНИЕ ОБРАЗОВАНИЯ МЭРИИ ГОРОДА ЧЕРЕПОВЦА,Определение поставщика отменено,/epz/order/notice/ea44/view/common-info.html?r...,разработка программного обеспечения
196,"Конкурс в электронной форме, участниками котор...",ЗП - 81023947 ВЫПОЛНЕНИЕ РАБОТ ПО РАЗРАБОТКЕ И...,Многолотовый заказРазмещено: 30.12.2019Обновле...,"Mon, 30 Dec 2019 09:26:17 GMT",223-ФЗ,"ПУБЛИЧНОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""СБЕРБАНК РОССИИ""",Закупка завершена,https://zakupki.gov.ru/223/purchase/public/pur...,разработка программного обеспечения
197,Прочие №32008781814,"Оказание услуг по сопровождению, информационно...",999516.09,"Mon, 20 Jan 2020 14:33:33 GMT",223-ФЗ,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕН...,Закупка завершена,https://zakupki.gov.ru/223/purchase/public/pur...,разработка программного обеспечения
198,Прочие №32008781485,Разработка электронных сервисов и настройка п...,102000.00,"Mon, 20 Jan 2020 13:35:51 GMT",223-ФЗ,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ОБРАЗОВА...,Закупка завершена,https://zakupki.gov.ru/223/purchase/public/pur...,разработка программного обеспечения
